In [ ]:
# ==========================================
# CH-SIMS V2 + V-JEPA 2: 兩階段 Fine-tune
# Google Colab 版（從 Drive 複製 Raw.zip）
# ==========================================

# ==========================================
# Step 1. 掛載 Google Drive
# ==========================================
!pip install -q "transformers[torch]" decord

from google.colab import drive
drive.mount('/content/drive')

# ==========================================
# Step 2. 從已掛載的 Google Drive 複製 Raw.zip 並解壓縮
# ==========================================
import os
import shutil
from tqdm import tqdm
import zipfile

def copy_with_progress(src, dst, buffer_size=1024*1024*16):
    total_size = os.path.getsize(src)
    with open(src, 'rb') as fsrc, open(dst, 'wb') as fdst:
        with tqdm(total=total_size, unit='B', unit_scale=True, desc="Copying zip") as pbar:
            while True:
                buf = fsrc.read(buffer_size)
                if not buf:
                    break
                fdst.write(buf)
                pbar.update(len(buf))

# ===== 設定路徑 =====
target_dir = '/content/datasets'
extract_dir = os.path.join(target_dir, 'CH-SIMS-V2')
os.makedirs(target_dir, exist_ok=True)

# 來源：掛載好的 Google Drive
src_path = '/content/drive/MyDrive/datasets/CH-SIMS-V2/Raw.zip'
dst_path = os.path.join(target_dir, 'Raw.zip')

# ===== 檢查來源檔案 =====
if not os.path.exists(src_path):
    raise FileNotFoundError(
        f"❌ 找不到來源檔案：{src_path}\n"
        "請確認 Raw.zip 已放在 MyDrive/datasets/CH-SIMS-V2/ 底下。"
    )

# ===== 從 Drive 複製到本地（/content/datasets）=====
if not os.path.exists(dst_path):
    print(f"📥 從 Google Drive 複製 Raw.zip 到 {dst_path} ...")
    copy_with_progress(src_path, dst_path)
    print("✅ 複製完成！")
else:
    print(f"⏩ {dst_path} 已存在，跳過複製。")

# 檢查檔案大小
if os.path.exists(dst_path):
    file_size_gb = os.path.getsize(dst_path) / (1024**3)
    print(f"檔案大小: {file_size_gb:.2f} GB")

# ===== 解壓縮 =====
if not os.path.exists(extract_dir):
    print(f"📂 解壓縮 {dst_path} 到 {extract_dir} ...")
    with zipfile.ZipFile(dst_path, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        print(f"壓縮檔包含 {len(file_list)} 個項目")
        for file in tqdm(file_list, desc="解壓縮中", unit="files"):
            zip_ref.extract(file, extract_dir)
    print("✅ 解壓縮完成！")
else:
    print(f"⏩ {extract_dir} 已存在，跳過解壓縮。")

print("\n📁 檢查資料結構...")

# 依據實際解壓結果，你之前的 log 顯示結構如下：
# /content/datasets/CH-SIMS-V2/ch-simsv2s/{meta.csv, Raw/}
root_level = os.listdir(extract_dir)
print("CH-SIMS-V2 底下內容：", root_level)

data_root_path = os.path.join(extract_dir, 'ch-simsv2s')
if not os.path.exists(data_root_path):
    print("❌ 找不到 ch-simsv2s，實際結構如下：")
    !ls -R {extract_dir} | head -n 80
    raise FileNotFoundError("請確認 zip 裡的資料夾名稱是否為 ch-simsv2s")

raw_folder = os.path.join(data_root_path, 'Raw')
if os.path.exists(raw_folder):
    video_folders = [d for d in os.listdir(raw_folder)
                     if os.path.isdir(os.path.join(raw_folder, d))]
    print(f"✅ 找到 Raw 資料夾：{raw_folder}")
    print(f"   共有 {len(video_folders)} 個影片資料夾")
    print("   範例資料夾：", video_folders[:10])
else:
    print("❌ 找不到 Raw 資料夾！")
    !ls -R {data_root_path} | head -n 80
    raise FileNotFoundError("Raw 資料夾不存在，請確認資料集壓縮檔結構")

# ===== 檢查 meta.csv =====
meta_csv_path = os.path.join(data_root_path, 'meta.csv')
if os.path.exists(meta_csv_path):
    print(f"✅ 找到 meta.csv：{meta_csv_path}")
else:
    print("❌ 找不到 meta.csv！")
    !find {extract_dir} -name "meta.csv"
    raise FileNotFoundError("meta.csv 未找到，請確認資料集結構")

print("\n" + "="*60)
print("資料準備完成！")
print("="*60)

# ===== 複製並解壓縮 bbox_cache =====
bbox_cache_name = "bbox_cache_merged"
bbox_cache_src = f'/content/drive/MyDrive/datasets/CH-SIMS-V2/{bbox_cache_name}.zip'
bbox_cache_local_zip = os.path.join(target_dir, f'{bbox_cache_name}.zip')
bbox_cache_dir = os.path.join(target_dir, 'CH-SIMS-V2', bbox_cache_name)

# 檢查是否有壓縮檔（如果 Drive 上是資料夾而非 zip，見下方替代方案）
if os.path.exists(bbox_cache_src):
    # 複製 zip 到本地
    if not os.path.exists(bbox_cache_local_zip):
        print(f"📥 複製 {bbox_cache_name}.zip 到本地...")
        shutil.copy2(bbox_cache_src, bbox_cache_local_zip)
        print("✅ 複製完成！")
    else:
        print(f"⏩ {bbox_cache_local_zip} 已存在，跳過複製。")

    # 解壓縮
    if not os.path.exists(bbox_cache_dir):
        print(f"📂 解壓縮 {bbox_cache_name}.zip...")
        with zipfile.ZipFile(bbox_cache_local_zip, 'r') as zip_ref:
            zip_ref.extractall(os.path.join(target_dir, 'CH-SIMS-V2'))
        print("✅ 解壓縮完成！")
    else:
        print(f"⏩ {bbox_cache_dir} 已存在，跳過解壓縮。")
else:
    # 如果 Drive 上是資料夾而非 zip，直接複製整個資料夾
    print("讀取舊的 bbox_cache，請確認 bbox_cache zip name")
    bbox_cache_folder_src = '/content/drive/MyDrive/datasets/CH-SIMS-V2/bbox_cache'

    if os.path.exists(bbox_cache_folder_src):
        if not os.path.exists(bbox_cache_dir):
            print(f"📥 複製 bbox_cache 資料夾到本地（這可能需要幾分鐘）...")
            shutil.copytree(bbox_cache_folder_src, bbox_cache_dir)
            print("✅ 複製完成！")
        else:
            print(f"⏩ {bbox_cache_dir} 已存在，跳過複製。")
    else:
        print("⚠️ 找不到 bbox_cache，將不使用 ASD 裁切")
        bbox_cache_dir = None

# 檢查 bbox_cache 內容
if bbox_cache_dir and os.path.exists(bbox_cache_dir):
    import glob
    cache_files = glob.glob(os.path.join(bbox_cache_dir, "**", "*.json"), recursive=True)
    print(f"✅ bbox_cache 包含 {len(cache_files)} 個 JSON 檔案")
else:
    print("⚠️ bbox_cache 不存在或為空")

print("\n" + "="*60)
print("資料準備完成！")
# print("="*60)

